In [3]:
from os import walk
import pandas as pd
import numpy as np
import tensorflow as tf

C:\Users\Dergel\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Dergel\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Dergel\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
data = []
for (dirpath, dirnames, filenames) in walk('./data/DREAM4_InSilico_Size10/'):
    for (dirpath1, dirnames1, filenames1) in walk(dirpath):
        for (dirpath2, dirnames2, filenames2) in walk(dirpath1):
            for f in filenames2:
                if '_timeseries' in f:
                    data.append(pd.read_table(dirpath2 + '/' + f))

In [83]:
SEQUENCE_LENGTH = 5

def DataGenerator():
    for d in data:
        cur_seq = []
        for _,row in d.iterrows():
            if row.Time == 0:
                if len(cur_seq) > 0:
                    cur_seq = np.array(cur_seq)
                    for i in range(SEQUENCE_LENGTH,cur_seq.shape[0] - SEQUENCE_LENGTH):
                        yield cur_seq[i-SEQUENCE_LENGTH:i], cur_seq[i]
                cur_seq = []
            cur_seq.append(row.drop('Time').values)

In [121]:
BATCH = 10

dataset = tf.data.Dataset.from_generator(DataGenerator, 
                               output_types=(tf.float32, tf.float32),
                               output_shapes=([width, 10], [10])
                               ).shuffle(500).batch(BATCH)

dataset.shuffle(BATCH)

<ShuffleDataset shapes: ((None, 5, 10), (None, 10)), types: (tf.float32, tf.float32)>

In [122]:
train = dataset.take(40)
test = dataset.skip(40)

In [125]:
inp = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH, 10), batch_size=BATCH)

hidden = tf.keras.layers.LSTM(20, return_sequences=False)(inp)

hidden = tf.keras.layers.Dense(10, activation='tanh')(hidden)
out = tf.keras.layers.Dense(10)(hidden)

model = tf.keras.models.Model(inputs=inp, outputs=out)
model.summary()

model.compile(optimizer='rmsprop', loss='mse')

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(10, 5, 10)]             0         
_________________________________________________________________
lstm_9 (LSTM)                (10, 20)                  2480      
_________________________________________________________________
dense_11 (Dense)             (10, 10)                  210       
_________________________________________________________________
dense_12 (Dense)             (10, 10)                  110       
Total params: 2,800
Trainable params: 2,800
Non-trainable params: 0
_________________________________________________________________


In [127]:
for i in range(10):
    history = model.fit(train, epochs=1)
    model.evaluate(test)
    print()

26/26 [==============================] - 1s 2ms/step - loss: 0.0164

26/26 [==============================] - 1s 2ms/step - loss: 0.0162

26/26 [==============================] - 1s 2ms/step - loss: 0.0158

26/26 [==============================] - 1s 2ms/step - loss: 0.0154

26/26 [==============================] - 1s 2ms/step - loss: 0.0146

26/26 [==============================] - 1s 2ms/step - loss: 0.0141

26/26 [==============================] - 1s 2ms/step - loss: 0.0138

26/26 [==============================] - 1s 2ms/step - loss: 0.0130

26/26 [==============================] - 1s 2ms/step - loss: 0.0133

26/26 [==============================] - 1s 2ms/step - loss: 0.0127

